# Polymers

1 a)

Ved å lagre informasjonen i et N x 2 array, vil kjøretiden minke betraktelig. Siden vi bare er interresert i posisjonen til de N monomerene, og ikke hele gitteret, vil dette være den beste framgangsmåten. Vi oppretter et array med dimensjon N x 2, der hver rad inneholder informasjon om posisjonen til en monomer i polymeren. Den første kolonnen inneholder x-koordinaten til polymeren, og den andre y-koordinaten. 

In [15]:
# Importing libraries
import numpy as np
import matplotlib.pyplot as plt

In [16]:
# 1b)Petar lukter

def createPolymer(n):
    """Inputs a number n and returns a horizontal polymer"""
    polymer = np.zeros((n,2))
    polymer[:,0] = np.linspace(0,n-1,n)
    polymer[:,1] = n//2
    return polymer

[[0. 2.]
 [1. 2.]
 [2. 2.]
 [3. 2.]
 [4. 2.]]


1c)
En rotatsjon om endepunktene vil ikke gi energiendring. Energiendring kommer av vekselvirkninger mellom monomerer som ligger inntil hverandre, men ikke er nabo-monomerer i selve polynomet.
Altså er det kun energiendring om enten som monomerer ligger i en avstand 1 unna hverandre flyttes fra hverandre, eller motsatt, hele tiden uten å være naboer i polynomet.

In [18]:
# 1e)

def isIntact(polymer, N):
    """Checks if a polymer is intact"""
    length = len(polymer)
    for punkt in polymer:
        if any(polymer) == punkt:
            return True
        else:
            return False
    if length == N:
        return True
    else:
        return False